# Converspational
Talk to your geospatial data

### Importing Packages

In [5]:
import os
import duckdb
from pathlib import Path
from openai import OpenAI
from pygame import mixer
from pvrecorder import PvRecorder
import wave, struct
from langchain.llms import Ollama
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import GPT4AllEmbeddings, OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain import hub, PromptTemplate
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain.chat_models import ChatOpenAI
from sqlalchemy import create_engine
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
from langchain.agents.agent_types import AgentType
from langchain.agents import create_sql_agent
from dotenv import load_dotenv

### Setting Keys

In [6]:
openai_api_key = os.getenv('OPENAI_API_KEY')
client = OpenAI(api_key=openai_api_key)

### Choose your LLM

#### Mistral 7B

In [ ]:
llm = Ollama(base_url='http://localhost:11434', model='mistral')

preset_embeddings = GPT4AllEmbeddings()

#### GPT 4

In [7]:
llm = ChatOpenAI(model_name="gpt-4", temperature=0)

preset_embeddings = OpenAIEmbeddings()

### Loading RAG data

#### Database (DuckDB)

##### Instantiate Database

In [8]:
duck_conn = duckdb.connect('./data/osopendata/os_opendata.db')
uri = 'duckdb:///./data/osopendata/os_opendata.db'
connect_args = {
    'read_only': True
}
sqlalchemy_conn = create_engine(uri)

##### Enable Spatial

In [9]:
duck_conn.query("INSTALL spatial;")
duck_conn.query("LOAD spatial;")

##### Import Data

In [10]:
duck_conn.query("CREATE SCHEMA IF NOT EXISTS raw;")
duck_conn.query("CREATE OR REPLACE TABLE raw.district_borough_unitary AS SELECT * FROM st_read('./data/osopendata/district_borough_unitary.gpkg');")
duck_conn.query("CREATE OR REPLACE TABLE raw.greenspace_site AS SELECT * FROM st_read('./data/osopendata/greenspace_site.gpkg');")

Investigate Data

In [11]:
duck_conn.query("DESCRIBE raw.district_borough_unitary")
duck_conn.query("DESCRIBE raw.greenspace_site")

┌────────────────────┬─────────────┬─────────┬─────────┬─────────┬───────┐
│    column_name     │ column_type │  null   │   key   │ default │ extra │
│      varchar       │   varchar   │ varchar │ varchar │ varchar │ int32 │
├────────────────────┼─────────────┼─────────┼─────────┼─────────┼───────┤
│ id                 │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ function           │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ distinctive_name_1 │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ distinctive_name_2 │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ distinctive_name_3 │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ distinctive_name_4 │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ geom               │ GEOMETRY    │ YES     │ NULL    │ NULL    │  NULL │
└────────────────────┴─────────────┴─────────┴─────────┴─────────┴───────┘

##### Create Database Agent

In [12]:

db = SQLDatabase.from_uri(
    uri,
    schema='raw',
    include_tables=['greenspace_site', 'district_borough_unitary'], 
	sample_rows_in_table_info=3)

template = """/
You are a SQL Analyst that is querying a database that contains two tables: One of greenspace polygons in Great Britain and the second of districts of Great Britain.

Both tables contain 'geom' columns of geometries in CRS:27700 which can be used to perform spatial joins and analysis using spatial functions commonly found in PostGIS.

Your job is to write and execute an accurate query that answers the following question:
{query}
"""

prompt = PromptTemplate.from_template(template)

toolkit = SQLDatabaseToolkit(
    db=db, 
    llm=llm)


agent_executor = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION)

/Users/rhargreaves/.pyenv/versions/3.9.13/envs/ged_chatbot/lib/python3.9/site-packages/duckdb_engine/__init__.py:162: DuckDBEngineWarning: duckdb-engine doesn't yet support reflection on indices
  warnings.warn(


#### Website

In [ ]:

loader = WebBaseLoader('https://en.wikipedia.org/wiki/Surfing')
data = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
all_splits = text_splitter.split_documents(data)

vectorstore = Chroma.from_documents(documents=all_splits, embedding=preset_embeddings)
retriever = vectorstore.as_retriever()

prompt = hub.pull("rlm/rag-prompt")

rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

##### Clean Vector Store

In [14]:
vectorstore.delete_collection()

### Record your question

#### Listening...

In [36]:
#load all audio input devices 
# for index, device in enumerate(PvRecorder.get_available_devices()):
#     print(f"[{index}] {device}")

recorder = PvRecorder(device_index=1, frame_length=512) #(32 milliseconds of 16 kHz audio)
audio = []
record_path = './audio/recording.mp3'

try:
    recorder.start()


    while True:
        frame = recorder.read()
        audio.extend(frame)
except KeyboardInterrupt:
    recorder.stop()
    with wave.open(record_path, 'w') as f:
        f.setparams((1, 2, 16000, 512, "NONE", "NONE"))
        f.writeframes(struct.pack("h" * len(audio), *audio))
finally:
    recorder.delete()

#### Speech-To-Text using OpenAI Whisper

In [41]:

audio_file = open(record_path, "rb")
transcript = client.audio.transcriptions.create(
  model="whisper-1", 
  file=audio_file, 
  response_format="text"
)

print(transcript)

Where did surfing originate from?



#### Alternatively input transcript text

In [28]:
transcript = "Where did Surfing originate from?"

### Question your data

#### Database

In [13]:
answer = agent_executor.run(
    prompt.format(query = "How many greenspaces with a function of 'Religious Grounds' are within 'Bath and North East Somerset'?")
)
print(answer)



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: "" 
Observation: district_borough_unitary, greenspace_site
Thought:The tables 'district_borough_unitary' and 'greenspace_site' seem to be the most relevant for this query. I should check their schemas to understand the structure and the fields they contain.
Action: sql_db_schema
Action Input: "district_borough_unitary, greenspace_site" 
Observation: 
CREATE TABLE raw.district_borough_unitary (
	"Name" VARCHAR, 
	"Area_Code" VARCHAR, 
	"Area_Description" VARCHAR, 
	"File_Name" VARCHAR, 
	"Feature_Serial_Number" INTEGER, 
	"Collection_Serial_Number" INTEGER, 
	"Global_Polygon_ID" INTEGER, 
	"Admin_Unit_ID" INTEGER, 
	"Census_Code" VARCHAR, 
	"Hectares" DOUBLE PRECISION, 
	"Non_Inland_Area" DOUBLE PRECISION, 
	"Area_Type_Code" VARCHAR, 
	"Area_Type_Description" VARCHAR, 
	"Non_Area_Type_Code" VARCHAR, 
	"Non_Area_Type_Description" VARCHAR, 
	geom BYTEA
)

/*
3 rows from district_borough_unitary table:
Name	Ar

#### Website

In [28]:
answer = rag_chain.invoke(transcript)
print(answer)

There are 79 greenspaces with a function of 'Religious Grounds' within 'Bath and North East Somerset'.


### Hear your answer

#### Text-To-Speech using OpenAI Whisper

In [14]:
speech_path = Path().parent / "./audio/speech.mp3"
response = client.audio.speech.create(
  model="tts-1",
  voice="fable",
  input=answer
)

response.stream_to_file(speech_path)

mixer.init()
mixer.music.load(speech_path)
mixer.music.play()